In [1]:
# multi headed multi-step cnn
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Model
from keras.layers import Input
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt

from func1 import *
from query1 import *
from Func_Paper1 import *

Using TensorFlow backend.


In [2]:
df = data1(30)

df.index = df['datetime']
df.index = df.index.astype('datetime64[ns]')
df.sort_index(inplace=True)
df.dropna(inplace=True)
df['datetime'] = pd.to_datetime(df['datetime'])
print("shape ", df.shape)

shape  (30, 33)


In [3]:
df.tail(1)

,datetime,fore24,tail24,tail24_avg,evap,infl,losses,rel1,rel2,rel3,...,run_c1,run_c2,run_c3,spillway,irr,camp,demand,derate,outage,stor
datetime,,,,,,,,,,,,,,,,,,,,,
2020-06-03,2020-06-03,139.71,82.5,82.18,0.5655,6.3569,0.0,4.6418,4.7447,3.6625,...,0.8,85.1,100.3,0.0,0.0,8489.0,13.0,75.0,0.0,4046.9548


In [4]:
dtat0 = df.iloc[-1:,0:1].values[0][0]
dti = pd.date_range(dtat0, periods=30).shift(1, freq='D')
dfdt = dti.to_frame(index=False, name='datims')

In [5]:
# dfdt

In [6]:
df.drop(('datetime'), axis=1, inplace=True)

In [7]:
from sklearn.preprocessing import MinMaxScaler
y_scaler = MinMaxScaler()
y_scaler.fit(df[['stor']])
X_scaler = MinMaxScaler()

df[['fore24', 'tail24', 'tail24_avg', 'evap', 'infl', 'losses',
       'rel1', 'rel2', 'rel3', 'rel_tol', 'engr1', 'engr2', 'engr3', 'cond1',
       'cond2', 'cond3', 'str1', 'str2', 'str3', 'run_g1', 'run_g2', 'run_g3',
       'run_c1', 'run_c2', 'run_c3', 'spillway', 'irr', 'camp', 'demand',
       'derate', 'outage', 'stor']] = X_scaler.fit_transform(df)

dfval = df.values[:,:-1]

In [8]:
def split_dataset(data):
    test = data
    test = array(split(test, len(test)/30))
    return test

In [9]:
test = split_dataset(df.values)

In [10]:
print('test', test.shape)

test (1, 30, 32)


In [11]:
n_input = 14

In [12]:
def forecast(model, history, n_input):

    data = array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))

    input_x = data[-n_input:, :]

    input_x = [input_x[:,i].reshape((1,input_x.shape[0],1)) for i in range(input_x.shape[1])]

    yhat = model.predict(input_x, verbose=1)

    yhat = yhat[0]
    
    return yhat

In [13]:
#เป็น list ที่สมาชิกแต่ละตัวเป็น samples (windows, n-feature)
history = [x for x in test]

In [14]:
print(len(history))
print(type(history))
print((array(history).shape))
print(type(array(history)))

1
<class 'list'>
(1, 30, 32)
<class 'numpy.ndarray'>


In [15]:
model = ShallowConvNet(test, n_input)
# model.summary()
model.load_weights("models/weights27.hdf5")
model.compile(loss='mse', optimizer='adam')

In [16]:
predictions = list()
for i in range(len(test)):
    # predict the week
    yhat_sequence = forecast(model, history, n_input)
    # store the predictions
    predictions.append(yhat_sequence)
    # get real observation and add to history for predicting the next week
#     history.append(test[i, :])
# evaluate predictions days for each week
predictions = array(predictions)

1/1 [==============================] - 1s 1s/step


In [17]:
yp = y_scaler.inverse_transform(predictions)

In [18]:
from sqlalchemy import create_engine
user='root'
pwd='opt#565784'
dbname='predictive'
host='10.211.1.25'
port=3306
con = create_engine(f'mysql+pymysql://{user}:{pwd}@{host}:{port}/{dbname}')

df = pd.DataFrame(yp).T
df.columns = ['stor30D']
com = pd.concat([dfdt, df], axis=1, join='inner')
com.to_sql('monthly', con, if_exists='replace', index=False)